<a href="https://colab.research.google.com/github/skywalker0803r/baseball_ProofofConcept/blob/main/%E9%AA%A8%E6%9E%B6%E6%B8%B2%E6%9F%93%E6%A8%A1%E7%B5%84.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# 影片轉骨架函數

In [2]:
import requests
import os

def convert_video_to_skeleton(video_path: str, api_url: str = 'https://mmpose-api-924124779607.us-central1.run.app/pose_video') -> dict:
    """
    將本地影片檔案發送至 API 進行姿勢偵測，並回傳骨架結果（JSON 格式）。

    Args:
        video_path (str): 本地影片檔案路徑，例如 'videos/pitch.mp4'
        api_url (str): 處理姿勢偵測的 API 端點 URL

    Returns:
        dict: 回傳的 JSON 結果，包含骨架資訊
    """
    if not os.path.exists(video_path):
        raise FileNotFoundError(f"找不到影片檔案：{video_path}")

    with open(video_path, 'rb') as f:
        files = {'file': (os.path.basename(video_path), f, 'video/mp4')}
        try:
            response = requests.post(api_url, files=files, timeout=120)
            response.raise_for_status()
            return response.json()
        except requests.exceptions.RequestException as e:
            print(f"❌ 錯誤：{e}")
            return {"error": str(e)}
result = convert_video_to_skeleton("/content/drive/MyDrive/Baseball Movies/CH_videos_4s/pitch_0001.mp4")
print(result)

{'frames': [{'frame_idx': 0, 'predictions': [[{'keypoints': [[626.0258178710938, 296.8128356933594], [624.4755249023438, 289.8363342285156], [622.1500244140625, 290.61151123046875], [598.89501953125, 292.93701171875], [610.5225219726562, 293.7121887207031], [606.6466674804688, 326.2691650390625], [593.4688720703125, 327.0443420410156], [594.2440185546875, 372.00396728515625], [586.4923706054688, 376.65496826171875], [627.576171875, 375.8797912597656], [625.2506713867188, 376.65496826171875], [577.9655151367188, 406.8864440917969], [590.3682250976562, 407.66162109375], [595.7943725585938, 485.1782531738281], [601.2205200195312, 485.95343017578125], [587.2675170898438, 556.4935302734375], [587.2675170898438, 561.14453125]], 'keypoint_scores': [0.7444220781326294, 0.6335139274597168, 0.6460358500480652, 0.4697836935520172, 0.6661820411682129, 0.5952520370483398, 0.7350718975067139, 0.4870814085006714, 0.7259783744812012, 0.5930870175361633, 0.6827380657196045, 0.46818020939826965, 0.68962

# 保存成 JSON 檔（推薦）

In [3]:
import json

def save_skeleton_to_json(data: dict, save_path: str = "skeleton_result.json"):
    with open(save_path, "w", encoding="utf-8") as f:
        json.dump(data, f, indent=2, ensure_ascii=False)
    print(f"✅ 已保存至 {save_path}")
save_skeleton_to_json(result, "CH_videos_4s_pitch_0001_skeleton.json")

✅ 已保存至 CH_videos_4s_pitch_0001_skeleton.json


# 影片渲染骨架函數 輸入影片路徑 骨架資料 回傳渲染好的影片路徑

In [11]:
import cv2

# COCO骨架連線索引，對應keypoints索引
COCO_CONNECTIONS = [
    (0, 1), (0, 2), (1, 3), (2, 4),      # 頭臉
    (5, 6),                             # 肩膀
    (5, 7), (7, 9),                     # 左手臂
    (6, 8), (8, 10),                    # 右手臂
    (5, 11), (6, 12), (11, 12),         # 軀幹臀部
    (11, 13), (13, 15),                 # 左腿
    (12, 14), (14, 16)                  # 右腿
]

def fast_render_pose_video(input_video_path: str, pose_json: dict, output_video_path: str):
    """
    將骨架資料渲染到影片並存檔，使用OpenCV，速度優先。

    Args:
        input_video_path (str): 原始影片路徑
        pose_json (dict): API回傳的骨架JSON資料
        output_video_path (str): 輸出影片路徑

    Returns:
        str: 輸出影片路徑
    """
    cap = cv2.VideoCapture(input_video_path)
    if not cap.isOpened():
        raise RuntimeError(f"無法開啟影片：{input_video_path}")

    width  = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
    height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
    fps    = cap.get(cv2.CAP_PROP_FPS)
    fourcc = cv2.VideoWriter_fourcc(*'mp4v')
    out = cv2.VideoWriter(output_video_path, fourcc, fps, (width, height))

    # 建立 frame_idx -> predictions 快速索引字典
    frames_data = {f['frame_idx']: f.get('predictions', []) for f in pose_json.get('frames', [])}

    frame_idx = 0
    while True:
        ret, frame = cap.read()
        if not ret:
            break

        predictions_raw = frames_data.get(frame_idx, [])

        # 處理多層巢狀list: 如果 predictions_raw 是 [[dict,...]], 取第一層
        if predictions_raw and isinstance(predictions_raw, list) and isinstance(predictions_raw[0], list):
            predictions = predictions_raw[0]
        else:
            predictions = predictions_raw

        # 畫骨架
        for person in predictions:
            keypoints = person.get('keypoints', [])
            scores = person.get('keypoint_scores', [])
            if not keypoints or not scores:
                continue

            # 畫點
            for i, (x, y) in enumerate(keypoints):
                if i < len(scores) and scores[i] > 0.3:
                    cv2.circle(frame, (int(x), int(y)), 4, (0, 255, 0), -1)

            # 畫骨架連線
            for (start, end) in COCO_CONNECTIONS:
                if start < len(keypoints) and end < len(keypoints):
                    x1, y1 = keypoints[start]
                    x2, y2 = keypoints[end]
                    if scores[start] > 0.3 and scores[end] > 0.3:
                        cv2.line(frame, (int(x1), int(y1)), (int(x2), int(y2)), (0, 255, 255), 2)

        out.write(frame)
        frame_idx += 1

    cap.release()
    out.release()
    return output_video_path
fast_render_pose_video('/content/drive/MyDrive/Baseball Movies/CH_videos_4s/pitch_0001.mp4', result, '/content/pitch_rendered.mp4')

'/content/pitch_rendered.mp4'